In [141]:
import pandas as pd
import numpy as np

#train
profile_data = pd.read_csv("/mnt/mainstore/data/QiWi/Social_Demography/qiwi_users_profile_data_train.csv", delimiter=';', encoding='cp1251', sep='\t')
trans_data_train = pd.read_csv("/mnt/mainstore/data/QiWi/Social_Demography/qiwi_money_out_data_train.csv", delimiter=';', encoding='cp1251', sep='\t')

#test
trans_data_test = pd.read_csv("/mnt/mainstore/data/QiWi/Social_Demography/qiwi_money_out_data_test.csv", delimiter=';', encoding='cp1251', sep='\t')
profile_data_test = pd.read_csv("/mnt/mainstore/data/QiWi/Social_Demography/qiwi_users_profile_data_test.csv", delimiter=';', encoding='cp1251', sep='\t')




In [233]:
payment_table = pd.merge(profile_data,trans_data_train, on='user_id', how = 'right')

payment_table_test = pd.merge(profile_data_test,trans_data_test, on='user_id', how = 'right')

set1 = set(payment_table.category)
set1 = [i for i in set1]
set1.sort()

set3.append('Файлообмен')
set3 = list(set(set3))

cat_dict = {one:index for index,one in enumerate(set3)}

univ = payment_table.university
univ = [i for i in univ]
setted_univ = set(univ)
setted_univ = [i for i in setted_univ]
setted_univ.sort()
#amount_univ = np.amount()

univ_dict = {one:index for index,one in enumerate(setted_univ)}

In [183]:
payment_table.columns

Index(['user_id', 'sex', 'age', 'university', 'graduation_year', 'category',
       'date_month', 'txn_count', 'payment'],
      dtype='object')

In [184]:
from collections import defaultdict

In [194]:
user_spendings = defaultdict(lambda: [0 for i in range(52)])
user_cats = defaultdict(lambda: [0 for i in range(52)])
for i, payment in payment_table.iterrows():
    user_spendings[payment['user_id']][cat_dict[payment['category']]] += payment['payment']
    user_cats[payment['user_id']][cat_dict[payment['category']]] += 1

In [195]:
for uid, value in user_cats.items():
    user_cats[uid] = np.array(value) / sum(value)

In [205]:
matrix = []
user_ids = list(user_cats.keys())
for uid in user_ids:
    matrix.append(np.concatenate([user_cats[uid], user_spendings[uid]]))

In [216]:
X = np.array(matrix)

In [234]:
Y_sex = (payment_table.sex.values[user_ids] == 'Муж').astype(int)
Y_age = payment_table.age.values[user_ids]

In [221]:
user_spendings_test = defaultdict(lambda: [0 for i in range(52)])
user_cats_test = defaultdict(lambda: [0 for i in range(52)])
for i, payment in payment_table_test.iterrows():
    user_spendings_test[payment['user_id']][cat_dict[payment['category']]] += payment['payment']
    user_cats_test[payment['user_id']][cat_dict[payment['category']]] += 1

In [222]:
for uid, value in user_cats_test.items():
    user_cats_test[uid] = np.array(value) / sum(value)

In [243]:
matrix_test = []
user_ids_test = list(user_cats_test.keys())
for uid in user_ids_test:
    matrix_test.append(np.concatenate([user_cats_test[uid], user_spendings_test[uid]]))

In [244]:
X_test = np.array(matrix_test)

In [266]:
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,GradientBoostingRegressor

#X_train, X_test1, y_train, y_test1 = cross_validation.train_test_split(X, Y, test_size=0.2)


#clf = RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=20)
clf = GradientBoostingClassifier(subsample=0.5, n_estimators=200, max_depth=12)
reg = GradientBoostingRegressor(subsample=0.5, n_estimators=200, max_depth=12)
clf.fit(X,Y_sex)
reg.fit(X,Y_age)
#clf.score(X_test1,y_test1)

sexes = clf.predict(X_test)
ages = reg.predict(X_test)

In [291]:
#ages.min()

sex1 = [i for i in sexes]

sex1.count(1)

15205

In [268]:
from collections import OrderedDict

In [269]:
answer = pd.DataFrame(OrderedDict(user_id=user_ids_test, sex=sexes, age=ages))[['user_id', 'sex', 'age']]

In [281]:
answer.sex = answer.sex.apply(lambda x: 'Жен' if x == 0 else 'Муж')
answer.age = answer.age.astype(int)

In [284]:
answer.to_csv('riteam_qiwi_2.csv', index=False)

In [286]:
!head riteam_qiwi_2.csv

user_id,sex,age
1,Муж,36
2,Муж,36
3,Муж,36
4,Муж,33
5,Муж,36
6,Муж,32
7,Муж,36
8,Муж,36
9,Муж,36


In [3]:

catigory_dict = {
 'IP-телефония':0,
 'MCMS':1,
 'MLM':2,
 'QVC':3,
 'QVP':4,
 'QVV':5,
 'Sim-карты для туристов':6,
 'VPP':7,
 'Абонентское обслуживание':8,
 'Авиабилеты':9,
 'Билеты в кино':10,
 'Билеты на зрелища':11,
 'Благотворительность':12,
 'Бронирование гостиниц':13,
 'Букмекеры':14,
 'Госуслуги':15,
 'Грузоперевозки и доставка':16,
 'Денежные переводы':17,
 'Ж/Д билеты':18,
 'ЖКУ':19,
 'Игры со ставками':20,
 'Интернет':21,
 'Интернет магазины':22,
 'Информационные услуги':23,
 'Коллекторские агентства':24,
 'Контент':25,
 'Купоны':26,
 'Лотереи':27,
 'Международная/междугородняя связь':28,
 'Местная связь':29,
 'Образовательные услуги':30,
 'Онлайн игры':31,
 'Онлайн общение':32,
 'Оплата подписки':34,
 'Охранные системы':35,
 'Переводы без открытия счета по свободным реквизитам':36,
 'Погашение кредитов':37,
 'Предоставление займов':38,
 'Радио':39,
 'Регистрация доменов':40,
 'Служебная':41,
 'Создание и дизайн сайтов, трафик':42,
 'Сотовая связь':43,
 'Страхование':44,
 'Такси':45,
 'Телевидение':46,
 'Транспортные карты':47,
 'Туристический продукт':48,
 'Форекс':49,
 'Хостинг':50,
 'Электронные деньги':51}

sex_dict = {
    'Муж':1,
    'Жен':-1
}

date_m = {
    '2016-06':6,
    '2016-07':7,
    '2016-08':8,
    '2016-09':9,
    '2016-10':10,
    '2016-11':11,
    '2016-12':12
}


In [117]:

from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


mlms = payment_table[payment_table.category == 'MLM']

#univ = payment_table[(payment_table.university == 'МГТУ им. Баумана')&(payment_table.sex == "Муж")]
#univ[univ.user_id == ]
univ = payment_table.university
univ = [i for i in univ]
setted_univ = set(univ)
setted_univ = [i for i in setted_univ]
setted_univ.sort()
#amount_univ = np.amount()

univ_dict = {one:index for index,one in enumerate(setted_univ)}

len(univ_dict)




#univ_name_and_pop
#univ


#features = [[catigory_dict[payment_table.category[i]],payment_table.payment[i],date_m[payment_table.date_month[i]]] for i in range(len(payment_table))]
#lables = [sex_dict[payment_table.sex[i]] for i in range(len(payment_table))]

#model = DecisionTreeClassifier()
#model.fit(features,lables)



#result = model.predict([43,300,10])
#mlms

#result


407

In [128]:
from sklearn import preprocessing,cross_validation
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
#from xgboost import XGBClassifier


features = [[cat_dict[payment_table.category[i]],univ_dict[payment_table.university[i]],payment_table.payment[i]] for i in range(len(payment_table))]
lables = [sex_dict[payment_table.sex[i]] for i in range(len(payment_table))]

features_test = [[cat_dict[payment_table_test.category[i]],univ_dict[payment_table_test.university[i]],payment_table_test.payment[i]] for i in range(len(payment_table_test))]  


X = np.array(features)
y = np.array(lables)

X = preprocessing.scale(X)
features_test = preprocessing.scale(features_test)



#print(len(X),len(y))

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2)

#clf = xgb.XGBClassifier(silent=False, nthread=4, max_depth=10, n_estimators=800, subsample=0.5, learning_rate=0.03)
#clf = RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=20)
#clf = GradientBoostingClassifier(subsample=0.5,max_depth=6)

#clf.fit(X_train,y_train)

#acc = clf.score(X_test,y_test)
acc

#result = clf.predict(features_test)

#print(result[2000:3000])
#ras_mass = [i for i in result]

#print(result[:1000])

#print('\n\n',ras_mass[:1000])
#print('women part: ', (ras_mass.count(-1)/len(res_mas))*100)

#print(ras_mass.count(-1))


/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, _DataConversionWarning)


0.81469338751385301